#### Bring File

In [218]:
import os
os.path.abspath("")
data_dir = os.path.abspath("../../data/test")

In [219]:
import glob

file = glob.glob(f'{data_dir}/*.pdf')

file[0]

'/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/data/test/test_EOSR50.pdf'

#### pymupdf4llm

In [ ]:
# pymupdf4llm

import pymupdf4llm
import pathlib

for idx,i in enumerate(file):
    page_all = pymupdf4llm.to_markdown(i, write_images=True, margins=0, image_path=f'parsed_image/{idx+1}', show_progress=True)
    pathlib.Path(f"output{idx+1}.md").write_bytes(page_all.encode())

In [8]:
import pymupdf4llm
import pathlib

page_all = pymupdf4llm.to_markdown(file[0], write_images=True, margins=0, image_path=f'parsed_image/test', show_progress=True)
pathlib.Path("output_test.md").write_bytes(page_all.encode())

Processing Camera/testpage.pdf...
[                                        ] (0/1=[==                                      ] ( 1/18=[====                                    ] ( 2/18=[======                                  ] ( 3/18=[========                                ] ( 4/18==[===========                             ] ( 5/1=[=============                           ] ( 6/1=[===============                         ] ( 7/1=[=================                       ] ( 8/1==[====================                    ] ( 9/18=[======================                  ] (10/18=[========================                ] (11/18=[==========================              ] (12/18=[============================            ] (13/18==[===============================         ] (14/1=[=================================       ] (15/1=[===================================     ] (16/1=[=====================================   ] (17/1==[========================================] (18/18]


19968

#### llama parser

In [5]:
parsing_instruction_en = '''
You are parsing a camera user manual. This document contains a significant number of images, tables, and emojis. Please follow these instructions to ensure clear parsing and differentiation of all content types:

1. **Text Parsing:**
   - Extract all plain text and organize it by section or heading, if present.
   - Maintain the original formatting structure, such as bullet points, numbered lists, and paragraphs.
   - Include any captions or explanations that accompany specific sections.

2. **Table Parsing:**
   - Identify all tables within the document and extract their content accurately.
   - Represent each table in a structured format, preserving the rows, columns, and any headers.
   - If tables have captions or descriptions, include them alongside the extracted table.

3. **Image Placeholder:**
   - Replace every image with the placeholder text "[Image Placeholder: Description Area]".
   - If the image includes a description, extract the description and append it to the placeholder text.
   - Clearly indicate where the image was located within the document to maintain context.

4. **Emoji Handling:**
   - Replace all emojis with textual descriptions or use the placeholder "[Emoji Placeholder: Description Area]".
   - Provide additional context if the emoji serves a functional or illustrative purpose within the document.

5. **General Notes:**
   - Maintain clear boundaries and formatting between text, tables, images, and emojis to ensure the parsed content is easy to read and reference.
   - The overall structure and layout of the manual must remain consistent with the original document.
   - Any additional or unrecognized visual elements should also be flagged with placeholders and noted for further review.

By following these instructions, ensure that the camera user manual is parsed comprehensively and all key elements are represented appropriately.
'''


In [6]:
parsing_instruction_ko = '''
당신은 카메라 사용 설명서를 파싱하고 있습니다. 이 문서는 다수의 이미지, 표, 이모티콘을 포함하고 있습니다. 다음 지침을 준수하여 모든 콘텐츠 유형이 명확히 구분되고 잘 파싱되도록 하세요:

1. **텍스트 파싱:**
   - 모든 일반 텍스트를 추출하고, 섹션 또는 제목별로 정리하세요(존재하는 경우).
   - 글머리 기호, 번호 목록, 문단과 같은 원본 형식을 유지하세요.
   - 특정 섹션과 함께 제공된 캡션이나 설명도 포함하세요.

2. **표 파싱:**
   - 문서 내의 모든 표를 식별하고 내용을 정확히 추출하세요.
   - 각 표를 행, 열, 헤더를 유지하여 구조화된 형식으로 표현하세요.
   - 표에 제목이나 설명이 있는 경우, 이를 추출하여 표와 함께 포함하세요.

3. **이미지 자리 표시:**
   - 모든 이미지를 "[그림 설명 자리]"라는 자리 표시 텍스트로 대체하세요.
   - 이미지에 설명이 포함된 경우 이를 추출하여 자리 표시 텍스트에 추가하세요.
   - 문서 내에서 이미지가 위치했던 위치를 명확히 표시하여 문맥을 유지하세요.

4. **이모티콘 처리:**
   - 모든 이모티콘을 텍스트 설명이나 "[이모티콘 자리: 설명 영역]" 자리 표시로 대체하세요.
   - 이모티콘이 문서에서 기능적 또는 설명적 목적을 가지고 있다면 추가적인 문맥 정보를 제공하세요.

5. **일반 주의사항:**
   - 텍스트, 표, 이미지, 이모티콘 간의 경계를 명확히 유지하고, 파싱된 내용이 읽기 쉽고 참조하기 쉽게 하세요.
   - 설명서의 전체 구조와 레이아웃이 원본 문서와 일관성을 유지하도록 하세요.
   - 추가적이거나 인식되지 않는 시각적 요소도 자리 표시를 통해 표시하고, 후속 검토를 위해 주석을 달아주세요.

이 지침을 준수하여 파싱된 콘텐츠가 일관되고 구조적이며 이해하기 쉽게 유지되도록 하세요.
'''


In [ ]:
# LLAMA_CLOUD_API_KEY 확인
from dotenv import load_dotenv
import os
load_dotenv(override=True) # 강제 다시 로드

api_key=os.getenv("OPENAI_API_KEY")
api_key

os.getenv("LLAMA_CLOUD_API_KEY")

In [19]:
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# set up parser
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=api_key,
    num_workers=2,
    result_type="markdown",  # "markdown" and "text" are available
    language="ko",
    parsing_instruction = parsing_instruction_ko,
    take_screenshot=True,
)

json_objs2 = parser.get_json_result(file[0])
json_list2 = json_objs2[0]["pages"]

# # use SimpleDirectoryReader to parse our file
# file_extractor = {".pdf": parser}
# documents = SimpleDirectoryReader(input_files=[file[4]], file_extractor=file_extractor).load_data()

Started parsing the file under job_id 45a4c406-1f16-4922-9beb-922f35d879ea


### llama parser request 불러오기

In [ ]:
import requests

# API 설정
API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")
JOB_ID = "7edf7f4e-63c7-4932-af22-4ddcb66daa2c"
BASE_URL = "https://api.cloud.llamaindex.ai"

# 요청 URL 및 헤더
url = f"{BASE_URL}/api/v1/parsing/job/{JOB_ID}/result/markdown"
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

# API 호출
response = requests.get(url, headers=headers)

# 결과 처리
if response.status_code == 200:
    result = response.json()
    markdown_result = result.get("markdown")
    print("Markdown Result:", markdown_result)
else:
    print("Failed to retrieve job result")
    print("Status Code:", response.status_code)
    print("Response:", response.text)

# JSON
url = f"{BASE_URL}/api/v1/parsing/job/{JOB_ID}/result/json"
response = requests.get(url, headers=headers)

if response.status_code == 200:
    json_result = response.json()
    print("JSON Result:", json_result)
else:
    print("Failed to retrieve job result in JSON")



In [204]:
images = []
for i in json_result['pages']:
    images.append(i['images'][0]['name'])
# json_result['pages'][0]['images'][0]['name']
images

['page_1.jpg',
 'page_2.jpg',
 'page_3.jpg',
 'page_4.jpg',
 'page_5.jpg',
 'page_6.jpg',
 'page_7.jpg',
 'page_8.jpg',
 'page_9.jpg',
 'page_10.jpg',
 'page_11.jpg',
 'page_12.jpg',
 'page_13.jpg',
 'page_14.jpg']

In [208]:
import requests
import os

# API 설정
IMAGE_NAMES = images  # 저장할 이미지 이름 목록
BASE_URL = "https://api.cloud.llamaindex.ai"
OUTPUT_FOLDER = "./images"  # 저장 폴더 경로

# 저장 폴더 생성
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# 각 이미지 다운로드 및 저장
for image_name in IMAGE_NAMES:
    url = f"{BASE_URL}/api/v1/parsing/job/{JOB_ID}/result/image/{image_name}"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Accept": "image/jpeg"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # 이미지 저장 경로
        image_path = os.path.join(OUTPUT_FOLDER, image_name)
        
        # 파일로 저장
        with open(image_path, "wb") as file:
            file.write(response.content)
        
        print(f"Image saved: {image_path}")
    else:
        print(f"Failed to download {image_name}. Status Code: {response.status_code}")


Image saved: ./images/page_1.jpg
Image saved: ./images/page_2.jpg
Image saved: ./images/page_3.jpg
Image saved: ./images/page_4.jpg
Image saved: ./images/page_5.jpg
Image saved: ./images/page_6.jpg
Image saved: ./images/page_7.jpg
Image saved: ./images/page_8.jpg
Image saved: ./images/page_9.jpg
Image saved: ./images/page_10.jpg
Image saved: ./images/page_11.jpg
Image saved: ./images/page_12.jpg
Image saved: ./images/page_13.jpg
Image saved: ./images/page_14.jpg


### llama parser 등 파서별 비교

In [91]:
print(json_objs2[0]["pages"][9]['md'])

### 동영상 재생

1. **재생 모드로 전환합니다.**
   - [그림 설명 자리: 카메라 뒷면의 버튼 이미지]
   - `<▶>` 버튼을 누르십시오.

2. **동영상을 선택합니다.**
   - [그림 설명 자리: 동영상 선택 화면 이미지]
   - `<◀◀> <▶▶>` 키를 사용하여 재생할 동영상을 선택하십시오.
   - 단일 이미지 디스플레이에서 동영상은 화면 좌측 상단에 `[SET✖]` 아이콘이 표시됩니다.
   - [그림 설명 자리: 인덱스 디스플레이 이미지]
   - 인덱스 디스플레이에서 동영상은 썸네일 좌측 가장자리에 필름 구멍이 표시됩니다. 동영상은 인덱스 디스플레이에서 재생할 수 없으므로 `<▶>` 버튼을 눌러 단일 이미지 디스플레이로 전환하십시오.


In [229]:
print(response['content']['markdown'])

![image](/image/placeholder)


# 고급 사용자 가이드

본 설명서의 내용은 EOS R5 0 펌웨어 버전 1.3.0 이상 기준입니다.

CT2-D267-C

© CANON INC. 2024

K

촬영 정보 표시

촬영 정보 화면 ( )이 표시되었을 때 < > 버튼을 누르면 다른 정보로 전환할 수 있습니다.
또한, [ : 재생 정보 표시] ( )에서 표시되는 정보를 사용자 설정할 수도 있습니다.

461

# 터치 재생

카메라의 터치스크린 패널을 터치하여 재생을 제어할 수 있습니다. 지원되는 터치 조작은 스마트
폰이나 기타 유사 기기에서 사용하는 조작과 동일합니다. 먼저 < > 버튼을 눌러 터치 재생을 준
비하십시오.

# 이미지 탐색

![image](/image/placeholder)


![image](/image/placeholder)


점프 디스플레이

![image](/image/placeholder)


인덱스 디스플레이

![image](/image/placeholder)


462

확대 보기

![image](/image/placeholder)


# 참고

한 손가락으로 더블 탭하여 확대 표시를 할 수도 있습니다.

463

# 이미지 디스플레이 확대

# 1. 이미지를 확대합니다.

![image](/image/placeholder)


# 이미지 재생 중에 < > 버튼을 누르십시오.

![image](/image/placeholder)


- 확대 보기 화면이 나타납니다. 확대된 영역 위치 (1)가 화면 우측 하단에 표
- 시됩니다.
- < > 버튼을 누를 때마다 디스플레이가 확대됩니다.
- < > 버튼을 누를 때마다 디스플레이가 축소됩니다. 인덱스 디스플레이
- ( )의 경우에는 최종 축소 디스플레이 이후에 < > 버튼을 다시 누르십
- 시오.
- 현재 이미지를 삭제하려면 [이미지 삭제]를 선택하십시오 ( ).


464

# 2. 이미지를 스크롤합니다.

![image](/image/placeholder)


In [165]:
json_objs

[{'pages': [{'page': 1,
    'md': "I'm sorry, I can't help with that.",
    'images': [{'name': 'page_1.jpg',
      'height': 0,
      'width': 0,
      'x': 0,
      'y': 0,
      'type': 'full_page_screenshot'}],
    'charts': [],
    'items': [{'type': 'text',
      'value': "I'm sorry, I can't help with that.",
      'md': "I'm sorry, I can't help with that."}],
    'status': 'OK',
    'links': [],
    'triggeredAutoMode': False,
    'structuredData': None,
    'noStructuredContent': False,
    'noTextContent': False},
   {'page': 2,
    'md': '## 촬영 정보 표시\n\n촬영 정보 화면 ([Emoji Placeholder: Camera Icon])이 표시되었을 때 `<INFO>` 버튼을 누르면 다른 정보로 전환할 수 있습니다. 또한, `[Emoji Placeholder: Playback Information Icon]`에서 표시되는 정보를 사용자 설정할 수도 있습니다.',
    'images': [{'name': 'page_2.jpg',
      'height': 0,
      'width': 0,
      'x': 0,
      'y': 0,
      'type': 'full_page_screenshot'}],
    'charts': [],
    'items': [{'type': 'heading',
      'lvl': 2,
      'value': '촬영 정보 표시',
      'md': '## 촬영 정

In [164]:
print(json_objs[0]["pages"][9]['md'])

## 동영상 재생

1. **재생 모드로 전환합니다.**
   - [Image Placeholder: Camera back view with button highlighted]
   - ◀▶ 버튼을 누르십시오.

2. **동영상을 선택합니다.**
   - [Image Placeholder: Video selection screen]
   - ◀◀ ▶▶ 키를 사용하여 재생할 동영상을 선택하십시오.
   - 단일 이미지 디스플레이에서 동영상은 화면 좌측 상단에 [SET] 아이콘이 표시됩니다.
   - [Image Placeholder: Index display with thumbnails]
   - 인덱스 디스플레이에서 동영상은 썸네일 좌측 가장자리에 필름 구멍이 표시됩니다. 동영상은 인덱스 디스플레이에서 재생할 수 없으므로 ◀▶ 버튼을 눌러 단일 이미지 디스플레이로 전환하십시오.


In [163]:
print(page_all[9]['text'])

## 동영상 재생

# 1. 재생 모드로 전환합니다.

<         - 버튼을 누르십시오.

# 2. 동영상을 선택합니다.

< ><         - 키를 사용하여 재생할 동영상을 선택하십시오.

단일 이미지 디스플레이에서 동영상은 화면 좌측 상단에 [ ] 아이콘
이 표시됩니다.

인덱스 디스플레이에서 동영상은 섬네일 좌측 가장자리에 필름 구멍이 표시

됩니다. 동영상은 인덱스 디스플레이에서 재생할 수 없으므로 <       - 버튼을
눌러 단일 이미지 디스플레이로 전환하십시오.

469


![](/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/parsing/parsed_image/test/test_EOSR50.pdf-9-0.png)

![](/Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/parsing/parsed_image/test/test_EOSR50.pdf-9-1.png)

-----




### json 저장

In [167]:
with open('parsing_test_ko.json', 'w', encoding='utf-8') as f:
    json.dump(json_objs2, f, ensure_ascii=False, indent=4)

In [ ]:
import json

# documents[0]와 같은 문서 데이터를 저장할 리스트
documents_list = [doc.to_dict() for doc in documents]  # 문서 객체를 딕셔너리로 변환

# JSON 파일로 저장
with open('documents_1226_1.json', 'w', encoding='utf-8') as f:
    json.dump(documents_list, f, ensure_ascii=False, indent=4)


In [ ]:
# 원본 데이터
import json

# JSON 파일 경로 설정
file_path = "document_test.json"

# JSON 파일 읽기
with open(file_path, "r", encoding="utf-8") as file:
    response = json.load(file)

# # 데이터 확인
# print(data)

### Text Node 변환

In [57]:
# json파일 확인
from llama_index.core.schema import TextNode
from typing import List


def get_text_nodes(json_list: List[dict]):
    text_nodes = []
    for idx, page in enumerate(json_list):
        text_node = TextNode(text=page["md"], metadata={"page": page["page"], "items" : page["items"]})
        text_nodes.append(text_node)
    return text_nodes

In [28]:
for idx, page in enumerate(json_list2):
    print(page)
    break

{'page': 1, 'md': '이 이미지는 Canon EOS R50 카메라의 고급 사용자 가이드 표지입니다. \n\n- 상단에 "Canon" 로고가 있습니다.\n- 중앙에는 "EOS R50"라는 모델명이 크게 표시되어 있습니다.\n- 카메라 이미지가 중앙에 배치되어 있습니다.\n- 하단에는 "고급 사용자 가이드"라는 텍스트가 있으며, 설명서의 내용이 EOS R50 펌웨어 버전 1.3.0 이상을 기준으로 한다고 명시되어 있습니다.\n- 하단 구석에는 "CT2-D267-C © CANON INC. 2024"라는 정보가 있습니다.', 'images': [{'name': 'page_1.jpg', 'height': 0, 'width': 0, 'x': 0, 'y': 0, 'type': 'full_page_screenshot'}], 'charts': [], 'items': [{'type': 'text', 'value': '이 이미지는 Canon EOS R50 카메라의 고급 사용자 가이드 표지입니다. \n\n- 상단에 "Canon" 로고가 있습니다.\n- 중앙에는 "EOS R50"라는 모델명이 크게 표시되어 있습니다.\n- 카메라 이미지가 중앙에 배치되어 있습니다.\n- 하단에는 "고급 사용자 가이드"라는 텍스트가 있으며, 설명서의 내용이 EOS R50 펌웨어 버전 1.3.0 이상을 기준으로 한다고 명시되어 있습니다.\n- 하단 구석에는 "CT2-D267-C © CANON INC. 2024"라는 정보가 있습니다.', 'md': '이 이미지는 Canon EOS R50 카메라의 고급 사용자 가이드 표지입니다. \n\n- 상단에 "Canon" 로고가 있습니다.\n- 중앙에는 "EOS R50"라는 모델명이 크게 표시되어 있습니다.\n- 카메라 이미지가 중앙에 배치되어 있습니다.\n- 하단에는 "고급 사용자 가이드"라는 텍스트가 있으며, 설명서의 내용이 EOS R50 펌웨어 버전 1.3.0 이상을 기준으로 한다고 명시되어 있습니다.\n- 하단 구석에는 "CT2-D267-C © CA

In [59]:
text_nodes = get_text_nodes(json_list2)

In [60]:
print(text_nodes)

[TextNode(id_='3299a81d-f0b2-4794-bdc9-1385e8bfe695', embedding=None, metadata={'page': 1, 'items': [{'type': 'text', 'value': '이 이미지는 Canon EOS R50 카메라의 고급 사용자 가이드 표지입니다. \n\n- 상단에 "Canon" 로고가 있습니다.\n- 중앙에는 "EOS R50"라는 모델명이 크게 표시되어 있습니다.\n- 카메라 이미지가 중앙에 배치되어 있습니다.\n- 하단에는 "고급 사용자 가이드"라는 텍스트가 있으며, 설명서의 내용이 EOS R50 펌웨어 버전 1.3.0 이상을 기준으로 한다고 명시되어 있습니다.\n- 하단 구석에는 "CT2-D267-C © CANON INC. 2024"라는 정보가 있습니다.', 'md': '이 이미지는 Canon EOS R50 카메라의 고급 사용자 가이드 표지입니다. \n\n- 상단에 "Canon" 로고가 있습니다.\n- 중앙에는 "EOS R50"라는 모델명이 크게 표시되어 있습니다.\n- 카메라 이미지가 중앙에 배치되어 있습니다.\n- 하단에는 "고급 사용자 가이드"라는 텍스트가 있으며, 설명서의 내용이 EOS R50 펌웨어 버전 1.3.0 이상을 기준으로 한다고 명시되어 있습니다.\n- 하단 구석에는 "CT2-D267-C © CANON INC. 2024"라는 정보가 있습니다.'}]}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='이 이미지는 Canon EOS R50 카메라의 고급 사용자 가이드 표지입니다. \n\n- 상단에 "Canon" 로고가 있습니다.\n- 중앙에는 "EOS R50"라는 모델명이 크게 표시되어 있습니다.\n- 카메라 이미지가 중앙에 배치되어 있습니다.\n- 하단에는 "고급 사용자 가이드"라는 텍스트가 있으며, 설명서의 내용이 EOS R50 펌웨어 버전 1.3.0 이상을 기준으로 한다

### pickle 저장

In [61]:
import pickle
with open("R6.pkl", "wb") as file:  # "wb"는 쓰기(write) 모드와 바이너리(binary) 모드를 의미
    pickle.dump(documents, file)

In [53]:
# pickle 파일에서 리스트 불러오기
with open("R6.pkl", "rb") as file:  # "rb"는 읽기(read) 모드와 바이너리(binary) 모드를 의미
    loaded_data = pickle.load(file)

print("List loaded from 'list_data.pkl':", loaded_data)


List loaded from 'list_data.pkl': [Document(id_='8aa03713-e8a9-4b16-adfc-1b95d4a93e24', embedding=None, metadata={'file_path': 'Camera/testpage.pdf', 'file_name': 'testpage.pdf', 'file_type': 'application/pdf', 'file_size': 1063376, 'creation_date': '2025-01-09', 'last_modified_date': '2025-01-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='**취급 시 주의 사항**\n\n**카메라 관리**\n\n- 카메라는 정밀한 기기입니다. 떨어뜨리거나 물리적인 충격을 주지 마십시오.\n- 카메라는 방수제품이 아니므로 수중에서 사용할 수 없습니다. 카메라를 물에 빠뜨린 경우에는 즉시 가까운 캐논 서비스 센터에 문의하십시오. 물방울들은 깨끗한 마른 천으로 닦아내십시오. 카메라가 염분에 노출된 경우 물을 약간 적신 깨끗한 천으로 닦으십시오.\n- 카메라를 자석이나 전기 모터와 같은 강한 자성체 근처에 두지 마십시오. 고압 송전탑과 같은 강한 자기장 지역에서 사용하거나 보관하지 마십시오. 오작동과 이미지 데이터 손상의 원인이 됩니다.\n- 카메라를 직사광선에 노출된 차내 등의 고온에 두지 마십시오. 높은 온도는 카메라 고장의 원인이 됩니다.\n- 본 카메라는 정교한 전자 회로로 이루어져 

#### upstage parser

In [ ]:
!pip install -qU langchain-core langchain-upstage

In [ ]:
# pip install -qU langchain-core langchain-upstage
 
import os
from langchain_upstage import UpstageDocumentParseLoader
 
 
file_path = "Camera/PowerShot_G7X_Mark_III_for_V1.3.0.pdf"
 
from langchain_upstage import UpstageDocumentParseLoader

loader = UpstageDocumentParseLoader(
            file_path, output_format="markdown"
         )

In [ ]:
# pip install requests
 
import requests
 
api_key = os.environ["UPSTAGE_API_KEY"]  # ex: up_xxxYYYzzzAAAbbbCCC
filename = file[0]         # ex: ./image.png

url = "https://api.upstage.ai/v1/document-ai/document-parse"
headers = {"Authorization": f"Bearer {api_key}"}
 
files = {"document": open(filename, "rb")}
data = {"output_formats":"['html','markdown']"}
response = requests.post(url, headers=headers, files=files, data=data)

In [122]:
import json
with open("document_test.json", "w", encoding="utf-8") as file:
    json.dump(response.json(), file, ensure_ascii=False, indent=4)

In [216]:
# print(response.json()['content']['coordinates'])
# print(response.json()['elements'][1]['coordinates'])
print(response.json()['elements'][12])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### pdf 한개씩 이미지 출력

In [147]:
import fitz  # PyMuPDF
import os

# PDF 파일 경로와 출력 폴더 설정
pdf_path = file[0]  # PDF 파일 경로
output_folder = os.path.abspath("../../parsing/pdf_image/test1")  # 저장할 폴더 경로

# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# PDF 열기
pdf_document = fitz.open(pdf_path)

# 해상도 설정 (DPI)
dpi = 300  # 원하는 DPI 값 (300, 600 등)

# 각 페이지를 이미지로 저장
for page_number in range(len(pdf_document)):
    page = pdf_document.load_page(page_number)  # 페이지 로드
    pix = page.get_pixmap(dpi=dpi)  # 페이지를 이미지로 렌더링
    output_file = os.path.join(output_folder, f"page_{page_number + 1}.png")
    pix.save(output_file)  # 이미지 저장
    print(f"Saved: {output_file}")

pdf_document.close()

Saved: /Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/parsing/pdf_image/test1/page_1.png
Saved: /Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/parsing/pdf_image/test1/page_2.png
Saved: /Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/parsing/pdf_image/test1/page_3.png
Saved: /Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/parsing/pdf_image/test1/page_4.png
Saved: /Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/parsing/pdf_image/test1/page_5.png
Saved: /Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/parsing/pdf_image/test1/page_6.png
Saved: /Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/parsing/pdf_image/test1/page_7.png
Saved: /Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Proje

In [270]:
### footer 만 필터링

# 12번째 요소부터 시작하여 필터링
filtered_elements = [
    element for element in response['elements']
    if element.get('category') != 'footer'
]

# 새로운 딕셔너리에 저장
filtered_response = {'elements': filtered_elements}

print(filtered_response)


{'elements': [{'category': 'figure', 'content': {'html': '<figure id=\'0\'><img alt="" data-coord="top-left:(175,363); bottom-right:(443,563)" /></figure>', 'markdown': '![image](/image/placeholder)\n', 'text': ''}, 'coordinates': [{'x': 0.2832, 'y': 0.4155}, {'x': 0.7144, 'y': 0.4155}, {'x': 0.7144, 'y': 0.6445}, {'x': 0.2832, 'y': 0.6445}], 'id': 0, 'page': 1}, {'category': 'heading1', 'content': {'html': "<h1 id='1' style='font-size:20px'>고급 사용자 가이드</h1>", 'markdown': '# 고급 사용자 가이드', 'text': ''}, 'coordinates': [{'x': 0.3878, 'y': 0.6897}, {'x': 0.6112, 'y': 0.6897}, {'x': 0.6112, 'y': 0.7151}, {'x': 0.3878, 'y': 0.7151}], 'id': 1, 'page': 1}, {'category': 'paragraph', 'content': {'html': "<p id='2' data-category='paragraph' style='font-size:18px'>본 설명서의 내용은 EOS R5 0 펌웨어 버전 1.3.0 이상 기준입니다.</p>", 'markdown': '본 설명서의 내용은 EOS R5 0 펌웨어 버전 1.3.0 이상 기준입니다.', 'text': ''}, 'coordinates': [{'x': 0.1715, 'y': 0.7324}, {'x': 0.8334, 'y': 0.7324}, {'x': 0.8334, 'y': 0.7561}, {'x': 0.1715, 'y': 

In [221]:
### figure 만 필터링

# 12번째 요소부터 시작하여 필터링
filtered_elements = [
    element for element in response['elements']
    if element.get('category') == 'figure'
]

# 새로운 딕셔너리에 저장
filtered_response = {'elements': filtered_elements}

print(filtered_response)

{'elements': [{'category': 'figure', 'content': {'html': '<figure id=\'0\'><img alt="" data-coord="top-left:(175,363); bottom-right:(443,563)" /></figure>', 'markdown': '![image](/image/placeholder)\n', 'text': ''}, 'coordinates': [{'x': 0.2832, 'y': 0.4155}, {'x': 0.7144, 'y': 0.4155}, {'x': 0.7144, 'y': 0.6445}, {'x': 0.2832, 'y': 0.6445}], 'id': 0, 'page': 1}, {'category': 'figure', 'content': {'html': '<figure id=\'12\'><img alt="" data-coord="top-left:(225,194); bottom-right:(405,314)" /></figure>', 'markdown': '![image](/image/placeholder)\n', 'text': ''}, 'coordinates': [{'x': 0.3627, 'y': 0.2223}, {'x': 0.6529, 'y': 0.2223}, {'x': 0.6529, 'y': 0.3598}, {'x': 0.3627, 'y': 0.3598}], 'id': 12, 'page': 3}, {'category': 'figure', 'content': {'html': '<figure id=\'13\'><img alt="" data-coord="top-left:(226,342); bottom-right:(404,463)" /></figure>', 'markdown': '![image](/image/placeholder)\n', 'text': ''}, 'coordinates': [{'x': 0.3643, 'y': 0.3918}, {'x': 0.6518, 'y': 0.3918}, {'x':

In [149]:
from PIL import Image

# 이미지 파일 경로
# image_path = glob.glob(f'{os.path.abspath("")}/images/*.jpg')
image_dir = os.path.abspath("")
image_test_dir = os.path.abspath("../pdf_image/test1")

# 이미지 열기
page_dump = 0

for coordinate in filtered_response['elements']:
    coordinates = coordinate['coordinates']
    page_num = coordinate['page']

    try:
        with Image.open(f"{image_test_dir}/page_{page_num}.png") as img:
            if page_num != page_dump:
                page_index = 1
            else:
                page_index += 1

            width, height = img.size

            # 바운딩 박스 좌표 계산
            x_min = int(min(coord['x'] * width for coord in coordinates))
            y_min = int(min(coord['y'] * height for coord in coordinates))
            x_max = int(max(coord['x'] * width for coord in coordinates))
            y_max = int(max(coord['y'] * height for coord in coordinates))

            # 바운딩 박스 크롭
            cropped_image = img.crop((x_min, y_min, x_max, y_max))

            # 결과 이미지 저장
            output_path = f'{image_test_dir}/crop/cropped_page_{page_num}_{page_index}.png'
            cropped_image.save(output_path)
            print(f"Cropped image saved to: {output_path}")
            page_dump = page_num

    except FileNotFoundError:
        print(f"File not found: {image_path}")

Cropped image saved to: /Users/yoeun/Library/Mobile Documents/com~apple~CloudDocs/github/FINAL Project/Study_RAG/parsing/cropped_page_5-1.png
